# Enhanced Flower Classification - Version 2.0

## 🎯 Key Improvements:
- **Advanced Data Augmentation** - More diverse transformations
- **Class Imbalance Handling** - Weighted training for balanced learning
- **Enhanced Visualizations** - Better dataset insights
- **Improved Models** - Optimized architectures with regularization

### Version 1 Results (Baseline):
- Simple CNN: 54% (overfitting)
- VGG16: 84% 
- MobileNetV2: 88% (best)

**Goal**: Achieve >90% accuracy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16, MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import os

print(f"TensorFlow: {tf.__version__}")
print("🚀 Enhanced System Ready")

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Enhanced Configuration
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
EPOCHS = 20
LEARNING_RATE = 0.0001
VALIDATION_SPLIT = 0.2

TRAIN_DIR = "archive_2/train"
TEST_DIR = "archive_2/test"

## 📊 Enhanced Dataset Analysis with Class Imbalance Detection

In [ ]:
# Analyze class distribution and detect imbalance
def analyze_dataset(data_dir):
    class_counts = {}
    for class_name in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_name)
        if os.path.isdir(class_path):
            count = len([f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            class_counts[class_name] = count
    
    counts = list(class_counts.values())
    imbalance_ratio = max(counts) / min(counts) if min(counts) > 0 else 0
    
    print(f"Total: {sum(counts):,} | Classes: {len(class_counts)} | Imbalance: {imbalance_ratio:.2f}")
    if imbalance_ratio > 1.5:
        print("⚠️ Imbalance detected - applying class weights")
    
    return class_counts, imbalance_ratio

train_classes, train_imbalance = analyze_dataset(TRAIN_DIR)
test_classes, _ = analyze_dataset(TEST_DIR)
class_names = list(train_classes.keys())

# Enhanced visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('🎯 Enhanced Dataset Analysis', fontweight='bold')

# Class distribution
train_counts = [train_classes[name] for name in class_names]
test_counts = [test_classes[name] for name in class_names]
x = range(len(class_names))

axes[0].bar([i-0.2 for i in x], train_counts, 0.4, label='Train', alpha=0.8)
axes[0].bar([i+0.2 for i in x], test_counts, 0.4, label='Test', alpha=0.8)
axes[0].set_title('Class Distribution')
axes[0].set_xticks(x)
axes[0].set_xticklabels(class_names, rotation=45)
axes[0].legend()

# Imbalance visualization
max_count = max(train_counts)
imbalance_factors = [max_count/count for count in train_counts]
axes[1].bar(class_names, imbalance_factors, color=sns.color_palette("viridis", len(class_names)))
axes[1].set_title('Class Imbalance Factors')
axes[1].set_xticklabels(class_names, rotation=45)
axes[1].axhline(y=1.5, color='red', linestyle='--', alpha=0.7, label='Threshold')
axes[1].legend()

plt.tight_layout()
plt.show()

## ⚖️ Class Imbalance Handling

In [ ]:
# Calculate class weights for balanced training
def calculate_class_weights(class_counts):
    classes = list(class_counts.keys())
    counts = np.array(list(class_counts.values()))
    
    classes_sklearn = np.arange(len(classes))
    sample_weights = np.repeat(classes_sklearn, counts)
    weights = compute_class_weight('balanced', classes=classes_sklearn, y=sample_weights)
    
    class_weights_dict = {i: weights[i] for i in range(len(classes))}
    
    print("⚖️ Class Weights:")
    for i, class_name in enumerate(classes):
        print(f"{class_name}: {weights[i]:.3f}")
    
    return class_weights_dict

class_weights = calculate_class_weights(train_classes)

## 🎨 Advanced Data Augmentation

In [ ]:
# Create enhanced data generators with advanced augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    brightness_range=[0.6, 1.4],
    channel_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=VALIDATION_SPLIT
)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=VALIDATION_SPLIT)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create datasets
train_ds = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='training', shuffle=True, seed=42
)

val_ds = val_datagen.flow_from_directory(
    TRAIN_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='validation', shuffle=False, seed=42
)

test_ds = test_datagen.flow_from_directory(
    TEST_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

print(f"🔄 Advanced augmentation applied")
print(f"Train: {train_ds.samples} | Val: {val_ds.samples} | Test: {test_ds.samples}")

# 🏗️ Enhanced Model Architectures

In [ ]:
# Enhanced CNN with improved regularization
def create_enhanced_cnn():
    model = keras.Sequential([
        layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        
        # Block 1 with BatchNorm and Dropout
        layers.Conv2D(32, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(32, 3, activation='relu', padding='same'),
        layers.MaxPooling2D(2),
        layers.Dropout(0.25),
        
        # Block 2
        layers.Conv2D(64, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, 3, activation='relu', padding='same'),
        layers.MaxPooling2D(2),
        layers.Dropout(0.25),
        
        # Block 3
        layers.Conv2D(128, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        layers.Dropout(0.25),
        
        # Classifier with GlobalAveragePooling
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(5, activation='softmax')
    ])
    return model

# Enhanced VGG16
def create_enhanced_vgg16():
    base = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    base.trainable = False
    
    model = keras.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(5, activation='softmax')
    ])
    return model

# Enhanced MobileNetV2
def create_enhanced_mobilenet():
    base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    base.trainable = False
    
    model = keras.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(5, activation='softmax')
    ])
    return model

# Create and compile models
enhanced_cnn = create_enhanced_cnn()
enhanced_vgg16 = create_enhanced_vgg16()
enhanced_mobilenet = create_enhanced_mobilenet()

for model in [enhanced_cnn, enhanced_vgg16, enhanced_mobilenet]:
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

print("🏗️ Enhanced models created with improved architectures")

# 🎯 Training with Enhanced Techniques

In [ ]:
# Enhanced training with callbacks and class weights
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
]

print("🚀 Training Enhanced Models with Class Weights...")

# Train Enhanced CNN
print("\n1. Training Enhanced CNN...")
history_cnn = enhanced_cnn.fit(
    train_ds, epochs=EPOCHS, validation_data=val_ds,
    class_weight=class_weights, callbacks=callbacks, verbose=1
)

# Train Enhanced VGG16
print("\n2. Training Enhanced VGG16...")
history_vgg = enhanced_vgg16.fit(
    train_ds, epochs=EPOCHS, validation_data=val_ds,
    class_weight=class_weights, callbacks=callbacks, verbose=1
)

# Train Enhanced MobileNetV2
print("\n3. Training Enhanced MobileNetV2...")
history_mobile = enhanced_mobilenet.fit(
    train_ds, epochs=EPOCHS, validation_data=val_ds,
    class_weight=class_weights, callbacks=callbacks, verbose=1
)

print("\n✅ All enhanced models trained!")

# 📊 Enhanced Results & Performance Comparison

In [ ]:
# Visualize training results
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('🎯 Enhanced Models Performance', fontsize=16, fontweight='bold')

# Validation accuracy comparison
axes[0].plot(history_cnn.history['val_accuracy'], label='Enhanced CNN', linewidth=2)
axes[0].plot(history_vgg.history['val_accuracy'], label='Enhanced VGG16', linewidth=2)
axes[0].plot(history_mobile.history['val_accuracy'], label='Enhanced MobileNet', linewidth=2)
axes[0].set_title('Validation Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Validation loss comparison
axes[1].plot(history_cnn.history['val_loss'], label='Enhanced CNN', linewidth=2)
axes[1].plot(history_vgg.history['val_loss'], label='Enhanced VGG16', linewidth=2)
axes[1].plot(history_mobile.history['val_loss'], label='Enhanced MobileNet', linewidth=2)
axes[1].set_title('Validation Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Final test evaluation and comparison
def evaluate_model(model, model_name):
    test_loss, test_acc = model.evaluate(test_ds, verbose=0)
    print(f"{model_name}: {test_acc:.4f} ({test_acc*100:.2f}%)")
    return test_acc

print("🏆 FINAL TEST RESULTS:")
print("=" * 40)
cnn_acc = evaluate_model(enhanced_cnn, "Enhanced CNN      ")
vgg_acc = evaluate_model(enhanced_vgg16, "Enhanced VGG16    ")
mobile_acc = evaluate_model(enhanced_mobilenet, "Enhanced MobileNet")

print("\n📈 IMPROVEMENT ANALYSIS:")
print("=" * 50)
print("Model\t\t\tOriginal\tEnhanced\tGain")
print("-" * 50)
print(f"CNN\t\t\t54.0%\t\t{cnn_acc*100:.1f}%\t\t+{(cnn_acc*100-54.0):.1f}%")
print(f"VGG16\t\t\t84.0%\t\t{vgg_acc*100:.1f}%\t\t+{(vgg_acc*100-84.0):.1f}%")
print(f"MobileNetV2\t\t88.0%\t\t{mobile_acc*100:.1f}%\t\t+{(mobile_acc*100-88.0):.1f}%")

best_acc = max(cnn_acc, vgg_acc, mobile_acc)
best_model = ["Enhanced CNN", "Enhanced VGG16", "Enhanced MobileNet"][[cnn_acc, vgg_acc, mobile_acc].index(best_acc)]

print(f"\n🥇 Best Model: {best_model} with {best_acc*100:.2f}% accuracy")
print(f"🎯 Goal Achievement: {'✅ SUCCESS' if best_acc > 0.9 else '❌ CLOSE'} (Target: >90%)")

# Key improvements summary
print("\n🚀 KEY ENHANCEMENTS APPLIED:")
print("✅ Advanced data augmentation (8 techniques)")
print("✅ Class imbalance handling with weighted training")
print("✅ Improved model architectures with regularization")
print("✅ Enhanced training with callbacks")
print("✅ Better data visualization and analysis")